**Process:**
* Load and check data
* Data preprocess (2 datasets)
* Answer each question (7 in total)
 
**Notations:**

*stock.sas7bdat*
* PERMNO: a 5-digit firm identifier
* DATE: last trading day of the month, SAS format yymmddn8.
* COMNAM: company name
* EXCHCD: 1: NYSE, 2: AMEX, 3: NASDAQ
* SICCD: 4 digit SIC code
* PRC: closing price
* RET: monthly return
* SHROUT: shares outstanding in thousand shares

*ff3.sas7bdat*
* DATE: last trading day of the month
* SMB: small-minus-big return
* HML: high-minus-low return
* MKTRF: excess return on the market
* RF: risk-free rate
* UMD: momentum factor

In [1]:
# import packages
import pandas as pd
import numpy as np
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

# Load and check data

In [2]:
# load data
Stock = pd.read_sas("stock.sas7bdat")
RM = pd.read_sas("ff3.sas7bdat")

In [3]:
# show first few rows of Stock
Stock.head()

,PERMNO,DATE,COMNAM,EXCHCD,SICCD,PRC,RET,SHROUT
0,10001.0,2001-01-31,b'ENERGY WEST INC',3.0,4920.0,9.875,0.012821,2498.0
1,10001.0,2001-02-28,b'ENERGY WEST INC',3.0,4920.0,9.750,-0.012658,2504.0
2,10001.0,2001-03-30,b'ENERGY WEST INC',3.0,4920.0,10.000,0.038462,2509.0
3,10001.0,2001-04-30,b'ENERGY WEST INC',3.0,4920.0,9.750,-0.025000,2509.0
4,10001.0,2001-05-31,b'ENERGY WEST INC',3.0,4920.0,10.700,0.097436,2509.0


In [4]:
# show first few rows of RM
RM.head()

,DATE,SMB,HML,MKTRF,RF,UMD
0,2001-01-31 00:08:07.555199999,0.0657,-0.0490,0.0313,0.0054,-0.2506
1,2001-02-28 00:08:06.000000000,-0.0074,0.1290,-0.1005,0.0038,0.1251
2,2001-03-30 00:00:00.000000000,0.0034,0.0645,-0.0726,0.0042,0.0835
3,2001-04-30 00:08:07.382400000,0.0052,-0.0469,0.0794,0.0039,-0.0797
4,2001-05-31 00:00:00.000000000,0.0260,0.0314,0.0072,0.0032,0.0212


In [5]:
# select the columns we want
Stock = Stock[["DATE", "PERMNO", "PRC", "RET", "SHROUT"]]
RM = RM[["DATE", "MKTRF", "RF"]]

In [6]:
# check Stock's information
Stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895867 entries, 0 to 895866
Data columns (total 5 columns):
DATE      895867 non-null datetime64[ns]
PERMNO    895867 non-null float64
PRC       889427 non-null float64
RET       885676 non-null float64
SHROUT    894897 non-null float64
dtypes: datetime64[ns](1), float64(4)
memory usage: 34.2 MB


In [7]:
# check RM's information
RM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 3 columns):
DATE     204 non-null datetime64[ns]
MKTRF    204 non-null float64
RF       204 non-null float64
dtypes: datetime64[ns](1), float64(2)
memory usage: 4.9 KB


From the information above, we can find that DATE column in both datasets is already datetime object, other columns are float.

# Data preprocess

### First, deal with Stock

In [8]:
# select data after 2008/1/1
Stock = Stock[Stock.DATE > "2008-01-01"]

# create temporary Stock dataset betweeen 2008/01/01 and 2012/12/31
temp_Stock = Stock[Stock.DATE <= "2012/12/31"]

# create function to count observations for each company
def calculate_num(input_data, num):
    temp_data = input_data.sort_values("DATE", ascending=True)
    # find out last row of data's DATE, and transform it to string
    # beacuse we need to make sure companies still exist at 2012/12/31
    last_date = str(temp_data.iloc[-1, :]["DATE"].date())
    # create index to find non na data point according to RET
    na_index = temp_data.RET.isna() == False
    # select data without na
    temp_data = temp_data[na_index]
    # count observations from back to front
    # the observations need at least 36 data points
    if len(temp_data.iloc[-num:, ]) == num and last_date == "2012-12-31":
        return True
    else:
        return False
    
# group temp_Stock by "PERMNO" column, and apply "calculate_num" function to each group
count_result = temp_Stock.groupby("PERMNO").apply(lambda x: calculate_num(x, 36))

# save company's PERMNO if it met the requirement
target_PERMNO = count_result[count_result == True].index

# show the number of companies which meet the requirement
print("number of qualified companies:", len(target_PERMNO))

number of qualified companies: 3238


In [9]:
# transform target_PERMNO to set
target_PERMNO = set(list(map(lambda x:str(int(x)), target_PERMNO)))

# transform temp_Stock's PERMNO column to string
# in case of warnings, set a copy of temp_Stock
temp_Stock = temp_Stock.copy()
temp_Stock.PERMNO = temp_Stock.PERMNO.apply(lambda x: str(int(x)))

# create index to find whether temp_Stock's PERMNO is in target_PERMNO
selected_index = temp_Stock.PERMNO.apply(lambda x: x in target_PERMNO)

# select companies which meet the requirement
temp_Stock = temp_Stock[selected_index]

### Second, merge Stock with RM

In [10]:
# check the key we need while merging two dataframes
print("Stock's DATE format:", temp_Stock.DATE.iloc[0])
print("RM's DATE format:", RM.DATE.iloc[0])

Stock's DATE format: 2008-01-31 00:00:00
RM's DATE format: 2001-01-31 00:08:07.555199999


From the information above, we can find that RM's DATE format also contain hour, minute, and second. However, Stock's DATE format doesn't contain those information, so we need to unify their format.

In [11]:
# unify their DATE format by only selecting year, month, and day
# in case of warnings, set a copy of temp_Stock
temp_Stock = temp_Stock.copy()
temp_Stock["DATE"] = temp_Stock["DATE"].apply(lambda x:x.date())
RM["DATE"] = RM["DATE"].apply(lambda x:x.date())

# merge Stock with RM
Sample = temp_Stock.merge(RM, how="inner", on="DATE")

# create new column "RIRF" for excess return on market
Sample["RIRF"] = Sample["RET"] - Sample["RF"]

In [12]:
# define function to run regression
def run_regression(x, y):
    X = add_constant(x)
    Y = y
    model = OLS(Y, X, missing="drop")
    result = model.fit()
    params = pd.DataFrame(result.params).T
    t_value = pd.DataFrame(result.tvalues).T
    t_value.columns = ["cnost_t_value", "RIRF_t_value"]
    return params.join(t_value)

# group Sample by "PERMNO" column, and apply "run_regression" function to each group
reg_result = Sample.groupby("PERMNO").apply(lambda x: run_regression(x.MKTRF, x.RIRF))